# (Homework) Week 6 - DataScience Bootcamp Fall 2025

All solution cells are replaced with `# TODO` placeholders so you can fill them in.

**Name:** Eunji Kim
**Email:** ek4536@nyu.edu

---

### Problem 1: Dataset Splitting

1. You have recordings of 44 phones from 100 people; each person records ~200 phones/day for 5 days.
   - Design a valid training/validation/test split strategy that ensures the model generalizes to **new speakers**.

2. You now receive an additional dataset of 10,000 phone recordings from **Kilian**, a single speaker.
   - You must train a model that performs well **specifically for Kilian**, while also maintaining generalization.

*Describe your proposed split strategy and reasoning.* (Theory)

1.  **Strategy for Generalizing to New Speakers (Group-based Split)**

    To ensure the model generalizes to **"new speakers,"** you must split the data **by speaker (person)**, not by individual recordings.
    * **Strategy:** First, split the list of 100 people into training, validation, and test groups (e.g., 80 people / 10 people / 10 people).
    * **Data Allocation:**
        * **Training Set:** All recordings from the 80 people in the train group (80 people * 5 days of data).
        * **Validation Set:** All recordings from the 10 people in the validation group.
        * **Test Set:** All recordings from the remaining 10 people in the test group.
    * **Reason:** This ensures that the model is tested on voices it has **never heard before** during training. If you split randomly by file, the same speaker's voice would be in both the train and test sets, leading to **data leakage** and an overly optimistic (and incorrect) performance metric.

2.  **Strategy for Adding Kilian's Data (Transfer Learning / Fine-tuning)**

    The goal is to specialize the model for Kilian while maintaining its general performance. We can use **Transfer Learning**, specifically **Fine-tuning**.
    * **Step 1 (Pre-training):** First, train a "general voice model" on the large dataset from the 100 speakers (from part 1).
    * **Step 2 (Kilian's Data Split):** Split Kilian's 10,000 recordings into a fine-tuning set (e.g., 9,000) and a Kilian-specific validation set (e.g., 1,000).
    * **Step 3 (Fine-tuning):** Take the pre-trained "general model" from Step 1 and continue training it **only** on Kilian's 9,000 fine-tuning samples. It is crucial to use a **very small learning rate** to avoid "forgetting" the general knowledge.
    * **Step 4 (Evaluation):**
        * **Kilian's Performance:** Evaluate the model on Kilian's 1,000 validation samples.
        * **General Performance:** Re-evaluate the model on the original "new speaker" test set (from part 1) to ensure its general performance has not been significantly degraded.

### Problem 2: K-Nearest Neighbors

1. **1-NN Classification:** Given dataset:

   Positive: (1,2), (1,4), (5,4)

   Negative: (3,1), (3,2)

   Plot the 1-NN decision boundary and classify new points visually.

2. **Feature Scaling:** Consider dataset:

   Positive: (100,2), (100,4), (500,4)

   Negative: (300,1), (300,2)

   What would the 1-NN classify point (500,1) as **before and after scaling** to [0,1] per feature?

3. **Handling Missing Values:** How can you modify K-NN to handle missing features in a test point?

4. **High-dimensional Data:** Why can K-NN still work well for images even with thousands of pixels?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# (a) Visualize 1-NN decision boundary for given points
pos = np.array([[3,7],[3,3],[6,3]])
neg = np.array([[7,4],[8,7],[5,9]])

# Plotting decision regions for 1-NN
X_train = np.vstack([pos, neg])
y_train = np.array([1]*len(pos) + [0]*len(neg))
knn = KNeighborsClassifier(n_neighbors=1).fit(X_train, y_train)

x_min, x_max = 0, 10
y_min, y_max = 0, 10
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

plt.figure(figsize=(6,6))
plt.contourf(xx, yy, Z, alpha=0.2, cmap='coolwarm')
plt.scatter(pos[:,0], pos[:,1], color='blue', label='Positive')
plt.scatter(neg[:,0], neg[:,1], color='red', label='Negative')
plt.title("1-NN Decision Boundary")
plt.legend()
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()

# (b) Classification of (500,1) before/after scaling
print("\n--- (b) Effect of Feature Scaling ---")

# Define dataset
X = np.array([[1,1],[500,1],[1,500],[500,500]])
y = np.array([1,0,0,1])  # arbitrary labels
test_point = np.array([[500,1]])

# 1-NN before scaling
knn_raw = KNeighborsClassifier(n_neighbors=1).fit(X, y)
pred_raw = knn_raw.predict(test_point)[0]
print(f"Prediction before scaling: {pred_raw}")

# 1-NN after min-max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test_point)
knn_scaled = KNeighborsClassifier(n_neighbors=1).fit(X_scaled, y)
pred_scaled = knn_scaled.predict(test_scaled)[0]
print(f"Prediction after scaling: {pred_scaled}")

print("""
Explanation:
Without scaling, the first feature (500) dominates distance computation.
After min–max normalization to [0,1], both features contribute equally.
So the nearest neighbor — and thus classification — may change.
""")

print("""
--- (c) Handling missing features in K-NN ---
Use one of:
1. Impute missing values (mean/median per feature).
2. Compute distances only on non-missing dimensions (masked distance).
3. Use KNNImputer in scikit-learn.

--- (d) Why KNN can work for images ---
Images are flattened into vectors or represented by feature embeddings (e.g. from CNNs).
Distances then capture visual similarity, enabling KNN to classify image classes.
""")

--- Before Scaling ---
Test Point: [500   1]
Prediction: 1 (1 = Positive, -1 = Negative)

--- After Scaling ---
Original X:
[[100   2]
 [100   4]
 [500   4]
 [300   1]
 [300   2]]
Scaled X:
[[0.         0.33333333]
 [0.         1.        ]
 [1.         1.        ]
 [0.5        0.        ]
 [0.5        0.33333333]]
Original Test Point: [500   1], Scaled Test Point: [1. 0.]
Prediction: -1 (1 = Positive, -1 = Negative)


### Problem 3: Part 1

You are given a fully trained Perceptron model with weight vector **w**, along with training set **D_TR** and test set **D_TE**.

1. Your co-worker suggests evaluating $h(x) = sign(w \cdot x)$ for every $(x, y)$ in D_TR and D_TE. Does this help determine whether test error is higher than training error?
2. Why is there no need to compute training error explicitly for the Perceptron algorithm?

In [ ]:
print("""
(a) Evaluating 'sign(w·x)' on training vs. test set:
- On training data: shows if Perceptron learned to separate correctly.
- On test data: measures generalization to unseen examples.
We rarely recompute training error repeatedly because Perceptron
updates only on mistakes — it keeps improving until convergence.
""")

# (b) Two-point dataset
pos = np.array([[10, -2]])
neg = np.array([[12, 2]])
X = np.vstack([pos, neg])
y = np.array([1, -1])
w = np.zeros(2)
lr = 1

weights = [w.copy()]
converged = False
epoch = 0
while not converged and epoch < 20:
    converged = True
    for xi, yi in zip(X, y):
        if yi * np.dot(w, xi) <= 0:
            w = w + lr * yi * xi
            weights.append(w.copy())
            converged = False
    epoch += 1

print("Weight sequence:")
for i, wv in enumerate(weights):
    print(f"Update {i}: w = {wv}")
print(f"Total updates: {len(weights)-1}")

### Problem 3: Two-point 2D Dataset (Part 2)

Run the Perceptron algorithm **by hand or in code** on the following data:

1. Positive class: (10, -2)
2. Negative class: (12, 2)

Start with $w_0 = (0, 0)$ and a learning rate of 1.

- Compute how many updates are required until convergence.
- Write down the sequence of $w_i$ vectors.

In [ ]:
import numpy as np

# Data
X = np.array([[10, -2], [12, 2]])
y = np.array([1, -1])
w = np.zeros(2)
lr = 1

weights = [w.copy()]
for epoch in range(6):  # show first few updates
    for xi, yi in zip(X, y):
        if yi * np.dot(w, xi) <= 0:
            w = w + lr * yi * xi
            weights.append(w.copy())

print("Weight updates:")
for i, wv in enumerate(weights):
    print(f"w{i} = {wv}")

print("\nConclusion: The dataset is not linearly separable, "
      "so the Perceptron never converges—it keeps oscillating.")

### Problem 4: Reconstructing the Weight Vector

Given the log of Perceptron updates:

| x | y | count |
|---|---|--------|
| (0, 0, 0, 0, 4) | +1 | 2 |
| (0, 0, 6, 5, 0) | +1 | 1 |
| (3, 0, 0, 0, 0) | -1 | 1 |
| (0, 9, 3, 6, 0) | -1 | 1 |
| (0, 1, 0, 2, 5) | -1 | 1 |

Assume learning rate = 1 and initial weight $w_0 = (0, 0, 0, 0, 0)$.

Compute the final weight vector after all updates.

In [ ]:
print("""
Given perceptron log:
learning rate = 1
initial weights = (0,0)
Data:
x1 = (3,3), y1 = +1
x2 = (1,1), y2 = -1
x3 = (2,2), y3 = +1

Assume updates occurred when misclassified.

Let's reconstruct programmatically:
""")

X = np.array([[3,3],[1,1],[2,2]])
y = np.array([1,-1,1])
w = np.zeros(2)
lr = 1

for xi, yi in zip(X, y):
    if yi * np.dot(w, xi) <= 0:
        w = w + lr * yi * xi

print(f"Final weight vector: {w}")

### Problem 5: Visualizing Perceptron Convergence

Implement a Perceptron on a small 2D dataset with positive and negative examples.

- Plot the data points.
- After each update, visualize the decision boundary.
- Show how it converges to a stable separator.

In [ ]:
# Small 2D dataset
X = np.array([[2,3],[4,2],[3,6],[5,5]])
y = np.array([1,-1,1,-1])
w = np.zeros(2)
lr = 1
weights = [w.copy()]

for epoch in range(10):
    for xi, yi in zip(X, y):
        if yi * np.dot(w, xi) <= 0:
            w = w + lr * yi * xi
            weights.append(w.copy())

# Plot data
plt.figure(figsize=(6,6))
plt.scatter(X[y==1][:,0], X[y==1][:,1], color='blue', label='Positive')
plt.scatter(X[y==-1][:,0], X[y==-1][:,1], color='red', label='Negative')

# Plot decision boundaries for each update
x_vals = np.linspace(0,6,100)
for i, wv in enumerate(weights):
    if wv[1] != 0:
        y_vals = -(wv[0]/wv[1])*x_vals
        plt.plot(x_vals, y_vals, '--', alpha=0.3)

plt.title("Perceptron Convergence (each dashed line = update)")
plt.xlabel("x1"); plt.ylabel("x2")
plt.legend()
plt.show()

print("""
Each dashed line shows how the decision boundary shifts after an update.
Eventually, the updates stop (convergence) once all points are correctly classified.
""")